In this notebook different files from previous notebooks and ones downloaded separately are loaded, preprocessed and joined so that final analysis could be done on them. 

In [1]:
import pandas as pd 
import numpy as np 
import re
from pandasql import sqldf 

df_articles: contains the information for each article extracted from wikimedia

df_ores: contains the prediction for each article and its rev_id 

df_population: contains the population estimates for 2022 for each state 

df_states: contains the region wise division of each state

In [2]:
df_articles = pd.read_csv('page_record.csv')
df_ores = pd.read_csv('ores_predictions.csv')
df_population = pd.read_excel('NST-EST2022-POP.xlsx')
df_states = pd.read_excel('US States by Region - US Census Bureau.xlsx')

Preprocessing the population record dataframe. Primarily to correct the columns name issue as can be seen below and secondly there was a . before some names of states. So I removed those as we will be joining on these in future. 

In [3]:
df_population.head()

,table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Geographic Area,"April 1, 2020 Estimates Base",2020,2021,2022
1,United States,331449520,331511512,332031554,333287557
2,Northeast,57609156,57448898,57259257,57040406
3,Midwest,68985537,68961043,68836505,68787595
4,South,126266262,126450613,127346029,128716192


In [4]:
# Set the first row as column names
df_population.columns = df_population.iloc[0]

# Drop the first row (which is now used as column names)
df_population = df_population[1:]

# Reset the index (optional)
df_population.reset_index(drop=True, inplace=True)

# Display the DataFrame
df_population.tail()

,Geographic Area,"April 1, 2020 Estimates Base",2020,2021,2022
52,.Washington,7705247,7724031,7740745,7785786
53,.West Virginia,1793755,1791420,1785526,1775156
54,.Wisconsin,5893725,5896271,5880101,5892539
55,.Wyoming,576837,577605,579483,581381
56,.Puerto Rico,3285874,3281557,3262693,3221789


In [5]:
# Remove dots from the values in column 'A'
df_population['Geographic Area'] = df_population['Geographic Area'].str.replace('.', '', regex=False)

# Display the modified DataFrame
df_population.head()

,Geographic Area,"April 1, 2020 Estimates Base",2020,2021,2022
0,United States,331449520,331511512,332031554,333287557
1,Northeast,57609156,57448898,57259257,57040406
2,Midwest,68985537,68961043,68836505,68787595
3,South,126266262,126450613,127346029,128716192
4,West,78588565,78650958,78589763,78743364


In [6]:
df_population[df_population['Geographic Area'] == 'Alabama'][0:8]

,Geographic Area,"April 1, 2020 Estimates Base",2020,2021,2022
5,Alabama,5024356,5031362,5049846,5074296


Preprocessing the articles dataframe to separate the cities and states. A regex expression is written to ensure that state has an exact state and any entries before the state are just entered into the city column

In [7]:
df_articles.head()

,Unnamed: 0,pageid,ns,title,contentmodel,pagelanguage,pagelanguagehtmlcode,pagelanguagedir,touched,lastrevid,length,talkid,fullurl,editurl,canonicalurl,watchers,redirect,new
0,0,104730,0,"Abbeville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1171163550,24706,281244.0,"https://en.wikipedia.org/wiki/Abbeville,_Alabama",https://en.wikipedia.org/w/index.php?title=Abb...,"https://en.wikipedia.org/wiki/Abbeville,_Alabama",NaN,NaN,NaN
1,1,104761,0,"Adamsville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1177621427,18040,281272.0,"https://en.wikipedia.org/wiki/Adamsville,_Alabama",https://en.wikipedia.org/w/index.php?title=Ada...,"https://en.wikipedia.org/wiki/Adamsville,_Alabama",NaN,NaN,NaN
2,2,105188,0,"Addison, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1168359898,13309,281517.0,"https://en.wikipedia.org/wiki/Addison,_Alabama",https://en.wikipedia.org/w/index.php?title=Add...,"https://en.wikipedia.org/wiki/Addison,_Alabama",NaN,NaN,NaN
3,3,104726,0,"Akron, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1165909508,11710,281240.0,"https://en.wikipedia.org/wiki/Akron,_Alabama",https://en.wikipedia.org/w/index.php?title=Akr...,"https://en.wikipedia.org/wiki/Akron,_Alabama",NaN,NaN,NaN
4,4,105109,0,"Alabaster, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1179139816,20343,281444.0,"https://en.wikipedia.org/wiki/Alabaster,_Alabama",https://en.wikipedia.org/w/index.php?title=Ala...,"https://en.wikipedia.org/wiki/Alabaster,_Alabama",NaN,NaN,NaN


In [8]:
# Extract city and state from the 'Location' column
df_articles['City'] = df_articles['title'].str.split(',').str[0]
df_articles['State'] = df_articles['title'].str.split(',').str[-1]

In [9]:
df_articles[['City', 'State']] = df_articles[['City', 'State']].applymap(lambda x: x.strip())

In [10]:
#checking that how many unique values are in the states column
df_articles['State'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Utqiaġvik', 'Arkansas',
       'California', 'Los Angeles', 'San Diego', 'San Francisco',
       '2010 United States census', '2020 United States census',
       'Colorado', 'Denver', 'Delaware', 'Florida',
       'County (United States)', 'Miami', 'Georgia', 'Atlanta',
       'Echols County', 'Hawaii', 'Kauai', 'Idaho', 'Illinois', 'Indiana',
       'Indianapolis', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'New Orleans', 'Maine', 'Maryland', 'Massachusetts', 'Boston',
       'Nantucket', 'Michigan', 'Detroit', 'Minnesota', 'Minneapolis',
       'Mississippi', 'Missouri', 'St. Louis', 'Montana', 'New Hampshire',
       'Nevada', 'Las Vegas', 'New Jersey', 'New Mexico', 'New York',
       'Population', 'Square mile', 'New York City',
       'American National Standards Institute',
       'Federal Information Processing Standards',
       'Geographic Names Information System', 'Syracuse',
       'North Carolina', 'North Dakota', 'Ohio', 'Ok

df_articles and df_ores are joined on lastrevid and rev_id respectively and then a few columns are dropped. Rest all irrelevant columns will be dropped towards the end before saving the file.

In [11]:
join_1 = pd.merge(df_articles, df_ores, left_on=df_articles['lastrevid'],
                  right_on = df_ores['rev_id'], how='left')

In [12]:
join_1.columns

Index(['key_0', 'Unnamed: 0_x', 'pageid', 'ns', 'title', 'contentmodel',
       'pagelanguage', 'pagelanguagehtmlcode', 'pagelanguagedir', 'touched',
       'lastrevid', 'length', 'talkid', 'fullurl', 'editurl', 'canonicalurl',
       'watchers', 'redirect', 'new', 'City', 'State', 'Unnamed: 0_y',
       'rev_id', 'Prediction'],
      dtype='object')

In [13]:
join_1.drop(['key_0', 'Unnamed: 0_x'], axis = 1, inplace = True)

In [14]:
join_1[0:20]

,pageid,ns,title,contentmodel,pagelanguage,pagelanguagehtmlcode,pagelanguagedir,touched,lastrevid,length,...,editurl,canonicalurl,watchers,redirect,new,City,State,Unnamed: 0_y,rev_id,Prediction
0,104730,0,"Abbeville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1171163550,24706,...,https://en.wikipedia.org/w/index.php?title=Abb...,"https://en.wikipedia.org/wiki/Abbeville,_Alabama",NaN,NaN,NaN,Abbeville,Alabama,0.0,1.171164e+09,C
1,104761,0,"Adamsville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1177621427,18040,...,https://en.wikipedia.org/w/index.php?title=Ada...,"https://en.wikipedia.org/wiki/Adamsville,_Alabama",NaN,NaN,NaN,Adamsville,Alabama,1.0,1.177621e+09,C
2,105188,0,"Addison, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1168359898,13309,...,https://en.wikipedia.org/w/index.php?title=Add...,"https://en.wikipedia.org/wiki/Addison,_Alabama",NaN,NaN,NaN,Addison,Alabama,2.0,1.168360e+09,C
3,104726,0,"Akron, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1165909508,11710,...,https://en.wikipedia.org/w/index.php?title=Akr...,"https://en.wikipedia.org/wiki/Akron,_Alabama",NaN,NaN,NaN,Akron,Alabama,3.0,1.165910e+09,GA
4,105109,0,"Alabaster, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1179139816,20343,...,https://en.wikipedia.org/w/index.php?title=Ala...,"https://en.wikipedia.org/wiki/Alabaster,_Alabama",NaN,NaN,NaN,Alabaster,Alabama,4.0,1.179140e+09,C
5,104899,0,"Albertville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1179198677,26930,...,https://en.wikipedia.org/w/index.php?title=Alb...,"https://en.wikipedia.org/wiki/Albertville,_Ala...",34.0,NaN,NaN,Albertville,Alabama,5.0,1.179199e+09,C
6,105153,0,"Alexander City, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1179140073,25275,...,https://en.wikipedia.org/w/index.php?title=Ale...,"https://en.wikipedia.org/wiki/Alexander_City,_...",32.0,NaN,NaN,Alexander City,Alabama,6.0,1.179140e+09,GA
7,105086,0,"Aliceville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1167792390,31568,...,https://en.wikipedia.org/w/index.php?title=Ali...,"https://en.wikipedia.org/wiki/Aliceville,_Alabama",NaN,NaN,NaN,Aliceville,Alabama,7.0,1.167792e+09,GA
8,100811,0,"Allgood, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:35Z,1165909718,11278,...,https://en.wikipedia.org/w/index.php?title=All...,"https://en.wikipedia.org/wiki/Allgood,_Alabama",NaN,NaN,NaN,Allgood,Alabama,8.0,1.165910e+09,C
9,100812,0,"Altoona, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:35Z,1165909823,10679,...,https://en.wikipedia.org/w/index.php?title=Alt...,"https://en.wikipedia.org/wiki/Altoona,_Alabama",NaN,NaN,NaN,Altoona,Alabama,9.0,1.165910e+09,C


Over here our files from the above join are joined with df_population which has data of state wise population for 2022. A few columns are dropped

In [15]:
join_2 = pd.merge(join_1, df_population, left_on = join_1['State'],  right_on = df_population['Geographic Area'], 
                  how='left')

In [16]:
join_2.columns

Index([                       'key_0',                       'pageid',
                                 'ns',                        'title',
                       'contentmodel',                 'pagelanguage',
               'pagelanguagehtmlcode',              'pagelanguagedir',
                            'touched',                    'lastrevid',
                             'length',                       'talkid',
                            'fullurl',                      'editurl',
                       'canonicalurl',                     'watchers',
                           'redirect',                          'new',
                               'City',                        'State',
                       'Unnamed: 0_y',                       'rev_id',
                         'Prediction',              'Geographic Area',
       'April 1, 2020 Estimates Base',                           2020,
                                 2021,                           2022],
     

In [17]:
join_2.drop(['April 1, 2020 Estimates Base', 2020, 2021, 'key_0' , 'Unnamed: 0_y'],
            axis = 1, inplace = True)

In [18]:
join_2.head()

,pageid,ns,title,contentmodel,pagelanguage,pagelanguagehtmlcode,pagelanguagedir,touched,lastrevid,length,...,canonicalurl,watchers,redirect,new,City,State,rev_id,Prediction,Geographic Area,2022
0,104730,0,"Abbeville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1171163550,24706,...,"https://en.wikipedia.org/wiki/Abbeville,_Alabama",NaN,NaN,NaN,Abbeville,Alabama,1.171164e+09,C,Alabama,5074296.0
1,104761,0,"Adamsville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1177621427,18040,...,"https://en.wikipedia.org/wiki/Adamsville,_Alabama",NaN,NaN,NaN,Adamsville,Alabama,1.177621e+09,C,Alabama,5074296.0
2,105188,0,"Addison, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1168359898,13309,...,"https://en.wikipedia.org/wiki/Addison,_Alabama",NaN,NaN,NaN,Addison,Alabama,1.168360e+09,C,Alabama,5074296.0
3,104726,0,"Akron, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1165909508,11710,...,"https://en.wikipedia.org/wiki/Akron,_Alabama",NaN,NaN,NaN,Akron,Alabama,1.165910e+09,GA,Alabama,5074296.0
4,105109,0,"Alabaster, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1179139816,20343,...,"https://en.wikipedia.org/wiki/Alabaster,_Alabama",NaN,NaN,NaN,Alabaster,Alabama,1.179140e+09,C,Alabama,5074296.0


In [19]:
join_2.columns

Index([              'pageid',                   'ns',                'title',
               'contentmodel',         'pagelanguage', 'pagelanguagehtmlcode',
            'pagelanguagedir',              'touched',            'lastrevid',
                     'length',               'talkid',              'fullurl',
                    'editurl',         'canonicalurl',             'watchers',
                   'redirect',                  'new',                 'City',
                      'State',               'rev_id',           'Prediction',
            'Geographic Area',                   2022],
      dtype='object')

In [20]:
join_2.to_csv('join_with_states.csv')

In [21]:
df_states.head()

,REGION,DIVISION,STATE
0,Northeast,New England,Connecticut
1,Northeast,New England,Maine
2,Northeast,New England,Massachusetts
3,Northeast,New England,New Hampshire
4,Northeast,New England,Rhode Island


This is the final join with df_states to get the region and division which we will be using in our final anaylsis

In [22]:
join_3 = pd.merge(join_2, df_states, left_on = join_2['State'],  right_on = df_states['STATE'], 
                  how='left')


In [23]:
join_3.head()

,key_0,pageid,ns,title,contentmodel,pagelanguage,pagelanguagehtmlcode,pagelanguagedir,touched,lastrevid,...,new,City,State,rev_id,Prediction,Geographic Area,2022,REGION,DIVISION,STATE
0,Alabama,104730,0,"Abbeville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1171163550,...,NaN,Abbeville,Alabama,1.171164e+09,C,Alabama,5074296.0,South,East South Central,Alabama
1,Alabama,104761,0,"Adamsville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1177621427,...,NaN,Adamsville,Alabama,1.177621e+09,C,Alabama,5074296.0,South,East South Central,Alabama
2,Alabama,105188,0,"Addison, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1168359898,...,NaN,Addison,Alabama,1.168360e+09,C,Alabama,5074296.0,South,East South Central,Alabama
3,Alabama,104726,0,"Akron, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1165909508,...,NaN,Akron,Alabama,1.165910e+09,GA,Alabama,5074296.0,South,East South Central,Alabama
4,Alabama,105109,0,"Alabaster, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1179139816,...,NaN,Alabaster,Alabama,1.179140e+09,C,Alabama,5074296.0,South,East South Central,Alabama


In [24]:
join_3.columns

Index([               'key_0',               'pageid',                   'ns',
                      'title',         'contentmodel',         'pagelanguage',
       'pagelanguagehtmlcode',      'pagelanguagedir',              'touched',
                  'lastrevid',               'length',               'talkid',
                    'fullurl',              'editurl',         'canonicalurl',
                   'watchers',             'redirect',                  'new',
                       'City',                'State',               'rev_id',
                 'Prediction',      'Geographic Area',                   2022,
                     'REGION',             'DIVISION',                'STATE'],
      dtype='object')

In [25]:
join_3.columns

Index([               'key_0',               'pageid',                   'ns',
                      'title',         'contentmodel',         'pagelanguage',
       'pagelanguagehtmlcode',      'pagelanguagedir',              'touched',
                  'lastrevid',               'length',               'talkid',
                    'fullurl',              'editurl',         'canonicalurl',
                   'watchers',             'redirect',                  'new',
                       'City',                'State',               'rev_id',
                 'Prediction',      'Geographic Area',                   2022,
                     'REGION',             'DIVISION',                'STATE'],
      dtype='object')

ALL IRRELEVANT COLUMNS ARE NOW DROPPED TO BRING OUR TABLE TO DESIRED FORMAT. ANY MISSING VALUES ARE ALSO DROPPED 

In [26]:
join_3.drop(['pageid', 'ns', 'touched', 'length', 'talkid', 'fullurl', 'editurl', 'canonicalurl', 
             'watchers', 'redirect', 'new', 'City', 'lastrevid', 'Geographic Area', 'STATE', 'REGION', 'key_0',
            'contentmodel', 'pagelanguage', 'pagelanguagehtmlcode', 'pagelanguagedir'], 
            axis = 1, inplace = True)

In [27]:
join_3.head()

,title,State,rev_id,Prediction,2022,DIVISION
0,"Abbeville, Alabama",Alabama,1.171164e+09,C,5074296.0,East South Central
1,"Adamsville, Alabama",Alabama,1.177621e+09,C,5074296.0,East South Central
2,"Addison, Alabama",Alabama,1.168360e+09,C,5074296.0,East South Central
3,"Akron, Alabama",Alabama,1.165910e+09,GA,5074296.0,East South Central
4,"Alabaster, Alabama",Alabama,1.179140e+09,C,5074296.0,East South Central


In [28]:
final_df = join_3.dropna()

In [29]:
##Rows with missing values 
print(f'rows with missing values are: {join_3.shape[0] - final_df.shape[0]}')

rows with missing values are: 4942


In [30]:
final_df = final_df.rename(columns={2022: 'population'})

In [31]:
final_df.to_csv('wp_scored_city_articles_by_state.csv')